In [ ]:
import iota
from iota import *
import time
import ipyparallel
from iota.adapter.wrappers import RoutingWrapper

seed = b'SSFM9SBA9GIREICDJU9FVUKZUU9KLXUWQBIIFIYHXQRMAJQHUKCKRXWFJKDJG9DCHSOIEGH9' #random seed
uri = 'https://field.carriota.com:443' #carriota
api = Iota(uri, seed)

print(api.get_node_info())

In [ ]:
#parallel processing

# clients = ipyparallel.Client()
# dview = clients[:-1]
# print(len(dview), dview)
# dview.block = True
# with dview.sync_imports():
#     import time
#     from iota import Iota
#     from iota import ProposedTransaction
#     from iota import Tag
#     from iota import TryteString
#     from iota.adapter.wrappers import RoutingWrapper


In [ ]:
def spam(tx_num):

    broadcasts = []
    
    node_ip = 'localhost'
    
    api = Iota(
        RoutingWrapper('http://' + node_ip).add_route('attachToTangle', 'http://' + node_ip),
        seed = b'SSFM9SBA9GIREICDJU9FVUKZUU9KLXUWQBIIFIYHXQRMAJQHUKCKRXWFJKDJG9DCHSOIEGH9'
    )
    
    
    gna_result = api.get_new_addresses(count=2)
    addresses = gna_result['addresses']
    
    time_start = time.time()
    while True:
        
        print('starting at ' + time.ctime() + '...')
        proposed_tx = ProposedTransaction(
          # Recipient of the transfer.
            address = addresses[1],
            value = 0,
            tag = Tag(b'SPAMINGTON9999999999999999'),
            message = TryteString.from_unicode('sup'))
        
        print('proposed', time.time())
        tips = []
        tips = api.getTips()['hashes']
        print('got tips at ' + time.ctime())
        
        tx_to_approve = api.getTransactionsToApprove(depth=7)
        print('got transactions at ' + time.ctime())
        proposed_trunk = tx_to_approve['trunkTransaction']
        proposed_branch = tx_to_approve['branchTransaction']
        
        both_tips = False
        if proposed_trunk in tips and proposed_branch in tips:
            both_tips = True
        i = 0
        print('set tips at ' + time.ctime())
        
        
        if False: #tx_num > 0: #round(len(dview)*3/4):
            while not both_tips:
                tips = api.getTips()['hashes']
                tx_to_approve = api.getTransactionsToApprove(depth=7)
                trunk = tx_to_approve['trunkTransaction']
                branch = tx_to_approve['branchTransaction']
                if proposed_branch in tips and proposed_trunk not in tips:
                    if trunk in tips:
                        proposed_trunk = trunk
                        both_tips = True
                    elif branch in tips:
                        proposed_trunk = branch
                        both_tips = True
                elif proposed_branch not in tips and proposed_trunk in tips:
                    if trunk in tips:
                        proposed_branch = trunk
                        both_tips = True
                    elif branch in tips:
                        proposed_branch = branch
                        both_tips = True
                else:
                    if trunk in tips:
                        proposed_trunk = trunk
                        if branch in tips:
                            proposed_branch = branch
                            both_tips = True
                    elif branch in tips:
                        proposed_branch = branch
                        
                        
        print('found 2 tips after', i, 'iterations at ', time.ctime())
            
        proposed_tx.branch_transaction_hash = proposed_branch
        proposed_tx.trunk_transaction_hash = proposed_trunk

        trytes = api.prepare_transfer(transfers=[proposed_tx])['trytes']
        print('got trytes at ' +  time.ctime())
        attachment = api.attachToTangle(trunkTransaction=proposed_trunk,
                          branchTransaction=proposed_branch,
                          minWeightMagnitude=14,
                          trytes=trytes)
        print('attached at ' + time.ctime())
        broadcast = api.broadcastTransactions(trytes=attachment['trytes'])
        print('broadcasted at ' + time.ctime())
        
spam(1)
# dview.map(spam, range(len(dview)))